In [14]:
import cv2
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

In [52]:
images = glob.glob('../data/Nikon/1/*.JPG')
images.sort()
images

['../data/Nikon/1/DSC_1143.JPG',
 '../data/Nikon/1/DSC_1144.JPG',
 '../data/Nikon/1/DSC_1145.JPG',
 '../data/Nikon/1/DSC_1146.JPG',
 '../data/Nikon/1/DSC_1147.JPG',
 '../data/Nikon/1/DSC_1148.JPG',
 '../data/Nikon/1/DSC_1149.JPG',
 '../data/Nikon/1/DSC_1150.JPG',
 '../data/Nikon/1/DSC_1151.JPG',
 '../data/Nikon/1/DSC_1152.JPG',
 '../data/Nikon/1/DSC_1153.JPG',
 '../data/Nikon/1/DSC_1154.JPG',
 '../data/Nikon/1/DSC_1155.JPG',
 '../data/Nikon/1/DSC_1156.JPG',
 '../data/Nikon/1/DSC_1157.JPG',
 '../data/Nikon/1/DSC_1158.JPG',
 '../data/Nikon/1/DSC_1159.JPG',
 '../data/Nikon/1/DSC_1160.JPG',
 '../data/Nikon/1/DSC_1161.JPG',
 '../data/Nikon/1/DSC_1162.JPG',
 '../data/Nikon/1/DSC_1163.JPG',
 '../data/Nikon/1/DSC_1164.JPG',
 '../data/Nikon/1/DSC_1165.JPG',
 '../data/Nikon/1/DSC_1166.JPG',
 '../data/Nikon/1/DSC_1167.JPG',
 '../data/Nikon/1/DSC_1168.JPG',
 '../data/Nikon/1/DSC_1169.JPG',
 '../data/Nikon/1/DSC_1170.JPG',
 '../data/Nikon/1/DSC_1171.JPG',
 '../data/Nikon/1/DSC_1172.JPG',
 '../data/

In [53]:
img = cv2.imread(images[0], 0)

In [54]:
out = cv2.resize(img, (int(img.shape[1] / 4), int(img.shape[0] / 4)))

In [55]:
ret, corners = cv2.findChessboardCorners(img, (8,8))

In [56]:
print(corners)

[[[1884.1678   566.4466 ]]

 [[1888.6494   643.73334]]

 [[1893.0011   721.50165]]

 [[1897.7357   798.8255 ]]

 [[1902.2618   877.322  ]]

 [[1906.1576   955.2568 ]]

 [[1911.2668  1032.6003 ]]

 [[1915.6422  1110.7097 ]]

 [[1810.4375   569.0391 ]]

 [[1815.3989   647.48083]]

 [[1820.3832   724.59766]]

 [[1824.5215   803.4832 ]]

 [[1828.3557   882.3638 ]]

 [[1833.9661   960.5423 ]]

 [[1838.7622  1038.3391 ]]

 [[1842.5471  1117.342  ]]

 [[1736.4103   571.74664]]

 [[1742.3038   650.66345]]

 [[1745.8479   729.27405]]

 [[1750.3838   809.4202 ]]

 [[1755.8949   886.8694 ]]

 [[1759.3076   966.13635]]

 [[1764.8474  1043.5153 ]]

 [[1768.8676  1122.6852 ]]

 [[1662.023    576.4564 ]]

 [[1666.6616   653.6529 ]]

 [[1671.4677   734.26135]]

 [[1675.9814   812.31635]]

 [[1680.8726   891.71796]]

 [[1685.4868   970.4857 ]]

 [[1690.2743  1049.5846 ]]

 [[1694.526   1128.4242 ]]

 [[1587.4574   579.3146 ]]

 [[1591.657    658.08594]]

 [[1595.7317   737.41876]]

 [[1601.1097   816.7

In [ ]:
# termination criteria
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((11*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:11].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.
## images = glob.glob('*.jpg')
for fname in images:
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv.findChessboardCorners(gray, (7,11), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)
        corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
        imgpoints.append(corners)
        # Draw and display the corners
        cv.drawChessboardCorners(img, (7,11), corners2, ret)
        cv.imshow('img', img)
        cv.waitKey(500)
cv.destroyAllWindows()

In [13]:
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

In [14]:
ret


0.28506089018546465

In [15]:
objpoints


[array([[ 0.,  0.,  0.],
        [ 1.,  0.,  0.],
        [ 2.,  0.,  0.],
        [ 3.,  0.,  0.],
        [ 4.,  0.,  0.],
        [ 5.,  0.,  0.],
        [ 6.,  0.,  0.],
        [ 0.,  1.,  0.],
        [ 1.,  1.,  0.],
        [ 2.,  1.,  0.],
        [ 3.,  1.,  0.],
        [ 4.,  1.,  0.],
        [ 5.,  1.,  0.],
        [ 6.,  1.,  0.],
        [ 0.,  2.,  0.],
        [ 1.,  2.,  0.],
        [ 2.,  2.,  0.],
        [ 3.,  2.,  0.],
        [ 4.,  2.,  0.],
        [ 5.,  2.,  0.],
        [ 6.,  2.,  0.],
        [ 0.,  3.,  0.],
        [ 1.,  3.,  0.],
        [ 2.,  3.,  0.],
        [ 3.,  3.,  0.],
        [ 4.,  3.,  0.],
        [ 5.,  3.,  0.],
        [ 6.,  3.,  0.],
        [ 0.,  4.,  0.],
        [ 1.,  4.,  0.],
        [ 2.,  4.,  0.],
        [ 3.,  4.,  0.],
        [ 4.,  4.,  0.],
        [ 5.,  4.,  0.],
        [ 6.,  4.,  0.],
        [ 0.,  5.,  0.],
        [ 1.,  5.,  0.],
        [ 2.,  5.,  0.],
        [ 3.,  5.,  0.],
        [ 4.,  5.,  0.],


In [16]:
imgpoints



[array([[[1251.4984 ,  320.98956]],
 
        [[1247.9918 ,  406.17694]],
 
        [[1244.5929 ,  491.4348 ]],
 
        [[1240.7941 ,  576.5855 ]],
 
        [[1237.7365 ,  661.0019 ]],
 
        [[1234.3047 ,  745.5176 ]],
 
        [[1231.2028 ,  830.28827]],
 
        [[1163.3567 ,  324.43524]],
 
        [[1160.0219 ,  407.41125]],
 
        [[1156.7339 ,  490.68402]],
 
        [[1153.4651 ,  573.6736 ]],
 
        [[1150.4487 ,  656.28955]],
 
        [[1147.2737 ,  738.8162 ]],
 
        [[1144.6356 ,  821.13055]],
 
        [[1079.6342 ,  327.57922]],
 
        [[1076.1064 ,  408.70987]],
 
        [[1072.8969 ,  489.97812]],
 
        [[1069.8424 ,  571.2443 ]],
 
        [[1067.0505 ,  651.57404]],
 
        [[1064.1924 ,  732.04   ]],
 
        [[1061.4055 ,  812.5636 ]],
 
        [[ 999.09766,  330.42804]],
 
        [[ 996.2262 ,  410.10425]],
 
        [[ 993.26404,  489.3363 ]],
 
        [[ 990.0216 ,  568.6912 ]],
 
        [[ 987.4625 ,  647.2494 ]],
 
        [[ 9